In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time
import pyautogui
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import os 
import re

In [4]:
base_url = "https://www.grailed.com/designers/john-elliott"

# open up chrome. Don't use headless otherwise the feed-item doesn't fully load all the way
# headless = dont open up the chrome
#chrome_options.add_argument('--headless')
driver = webdriver.Chrome()
driver.get(base_url)

# wait 30 sec
timeout = 30

In [5]:
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='feed-item']")))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()


#Don't want the items in staff recommendations and other items, just want the ones that show up in the search
results = driver.find_elements(By.XPATH, '//div[@class="FiltersInstantSearch"]//div[@class="feed-item"]')

# just so i can see how many iterations i probably need. Should be 40
print(len(results))


pyautogui.keyDown('ctrl')
pyautogui.keyDown('shift')
pyautogui.press('c')
pyautogui.keyUp('ctrl')
pyautogui.keyUp('shift')

try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CLASS_NAME, '-listing-count')))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()

# Get the number of listings and turn it into a number to find out how many pages I need to scroll
ListingNumber = driver.find_element(By.XPATH, '//div[@class="-listing-count"]//span').text

ListingNumber=int(ListingNumber.split(" ")[0])

pyautogui.keyDown('ctrl')
pyautogui.keyDown('shift')
pyautogui.press('i')
pyautogui.keyUp('ctrl')
pyautogui.keyUp('shift')

# Number of Scrolls, add one in case the function rounds down
ScrollNumber=round(ListingNumber/len(results))+1
print(ScrollNumber)

# Start the scroll
for i in range(0,ScrollNumber):
    #results = driver.find_elements_by_xpath('//div[@class="feed-item"]')
    #Results=Results+results
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)

# Run again to see total number of results the scraper is going to get. Should be the same as ListingNumber above
results = driver.find_elements(By.CLASS_NAME, 'feed-item')
print(len(results))
print(ListingNumber)

40
100
1003
3972


In [6]:
dataList = []
# Start scraping. Notice there's a '.' in front of //div. It's so that you start searching in each feed-item post
# Certain items don't have some of these tags in them, so make a "try-except" statement 
for result in results:

    itemDataList = []
    try:
        name = result.find_element(By.XPATH, './/p[@data-cy="listing-title"]').text
        itemDataList.append(name)
    except NoSuchElementException:
        itemDataList.append("")

    try:
        new_price = result.find_element(By.XPATH, './/span[@data-testid="Current"]').text
        old_price = result.find_element(By.XPATH, './/span[@data-testid="Original"]').text
        itemDataList.append("")
        itemDataList.append(new_price)
        itemDataList.append(old_price)
        
    except NoSuchElementException:
        try:
            price = result.find_element(By.XPATH, './/span[@data-testid="Current"]').text
            itemDataList.append(price)
            itemDataList.append("")
            itemDataList.append("")
        except NoSuchElementException:
            itemDataList.append("")
            itemDataList.append("")
            itemDataList.append("")
    try:
        size = result.find_element(By.XPATH, './/p[@class="ListingMetadata_size__Ho8j4"]').text
        itemDataList.append(size)
    except NoSuchElementException:
        itemDataList.append("")

    try:
        curr_time=result.find_element(By.XPATH, ".//span[@class='ListingAge_dateAgo__JEsJV']").text
        itemDataList.append(curr_time)
    except NoSuchElementException:
        itemDataList.append("")
    try:
        last_bump=result.find_element(By.XPATH, ".//span[@class='ListingAge_strikeThrough__rrCHR']").text
        itemDataList.append(last_bump)
    except NoSuchElementException:
        itemDataList.append("")
    try:
        link = result.find_element(By.XPATH, './a').get_attribute("href")
        itemDataList.append(link)
    except NoSuchElementException:
        itemDataList.append("")
    dataList.append(itemDataList)
# Turn the lists into a DataFrame with some nice column names 
ItemDF=pd.DataFrame(dataList,columns=['Name','Price','NewPrice','OldPrice','Size','Time','LastBump','Link'])
print(ItemDF.head())

                                                Name Price NewPrice OldPrice  \
0               John Elliott Tie-Dye Long Sleeve Tee   $69                     
1        John Elliott oversized thermal crew sweater  $100                     
2                      Pre SS 19 Bogotá Bomber / OAK  $269                     
3                 BNWT John Elliott Paloma Hemi sz L  $250                     
4  John Elliott - THUMPER JACKET TYPE III- Presid...   $95                     

  Size                Time LastBump  \
0    L          2 days ago            
1    M           1 day ago            
2    L   about 3 hours ago            
3    L  about 17 hours ago            
4   XL          3 days ago            

                                                Link  
0  https://www.grailed.com/listings/59288126-japa...  
1  https://www.grailed.com/listings/59324684-john...  
2  https://www.grailed.com/listings/59363182-john...  
3  https://www.grailed.com/listings/59343882-john...  
4  https:

In [7]:
def photo_downloader(Folder_Name):
    chrome_options = webdriver.ChromeOptions()
    # headless:dont open up the chrome
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome()
    driver2 = webdriver.Chrome(options=chrome_options)
    # timeout after 30 sec
    timeout = 30

    # Lists to append everything to
    metaDataList = []
    ### Start Process. Link is a list of links from ItemDF
    for link in ItemDF['Link']:
        driver.get(link)

        images = driver.find_elements(By.CLASS_NAME, 'Thumbnails_thumbnail__ixtme')
        
        imageList = []
        i = 0
        for image in images:
            image_link = image.get_attribute("src")
            image_name = image.get_attribute("alt")
            image_name = re.sub('[^A-Za-z_ ]+', '', image_name)
            imageList.append(image_link)
            driver2.get(image_link)
            image = driver2.find_element(By.TAG_NAME, 'img')
            print(image_name)
            with open(os.path.join(Folder_Name, image_name + " " + str(i) + ".png"), 'wb') as file:
                file.write(image.screenshot_as_png)
            i+=1
        metaDataList.append(imageList)
    return metaDataList

In [8]:
chrome_options = webdriver.ChromeOptions()
# headless:dont open up the chrome
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome()
driver.get("https://www.grailed.com/designers/chrome-hearts")
# timeout after 30 sec
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='feed-item']")))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()


# Lists to append everything to
metaDataList = []
### Start Process. Link is a list of links from ItemDF
for link in ItemDF['Link']:

    driver.get(link)    
    itemData = []

    try:
        sold_price =driver.find_element(By.CLASS_NAME, "SoldPrice_soldPrice__3yy1H").text
        itemData.append(sold_price)
    except NoSuchElementException:
        itemData.append("$0")

    try:
        description=driver.find_element(By.CLASS_NAME, "Description_body__cJryj").text
        description = description.replace('\n', ', ')
        itemData.append(description)
    except NoSuchElementException:
        itemData.append(description)

    try:
        details =driver.find_elements(By.CLASS_NAME, "Details_detail__J0Uny")
        for detail in details:
            text = detail.text
            if "Color" in text:
                itemData.append(text[len("Color "):])
            if "Condition" in text:
                itemData.append(text[len("Condition "):])
    except NoSuchElementException:
        itemData.append("")
        itemData.append("")

    try:
        location = driver.find_element(By.CLASS_NAME, "Shipping_cost__EkgVa").text
        shipping_prices= {}
        dropDown = Select(driver.find_element(By.CLASS_NAME, "Select_input__WXr_8"))
        location = location[:location.find(" to")]
        location = location[location.rindex(" ")+1:]
        for i in range(7):
            dropDown.select_by_index(i)
            shipping_cost = driver.find_element(By.CLASS_NAME, 'Shipping_cost__EkgVa').text
            if "$" in shipping_cost:
                shipping_cost = shipping_cost[shipping_cost.find("$") : ]
                shipping_cost = shipping_cost[: shipping_cost.find(" ")]
            else:
                shipping_cost ="N/A"
            shipping_prices[dropDown.all_selected_options[0].text] = shipping_cost
        itemData.append(location)
        itemData.append(shipping_prices)
        
    except NoSuchElementException:
        itemData.append("")
        itemData.append("")
    # itemData.append(photo_downloader("Images"))
    itemData.append("")
    print(itemData)
    metaDataList.append(itemData)


# Turn it into DataFrame
dataDF=pd.DataFrame(metaDataList,
columns=['SoldPrice', 'Description', 'Color', 'Condition', 'ShippingPrices','ImageLinks'])
dataDF

['', "John Elliott Tie-Dye Long Sleeve Tee, Size on tag 2, Real size L-Xl, Please check measurements ‼️ Please see all the photo ‼️, Do you have a question?I'm waiting in dm!, Have a nice day!", 'Beige', 'Gently Used', 'Europe', {'United States': '$10', 'United Kingdom': '$10', 'Asia': '$10', 'Canada': '$15', 'Europe': '$10', 'Australia/NZ': '$15', 'Other': '$15'}, '']
['', 'Size M, washed olive color, ribbed collar and cuffs, raglan sleeves. Soft vintage look and feel.', 'Washed olive', 'Gently Used', 'US', {'United States': '$15', 'United Kingdom': 'N/A', 'Asia': 'N/A', 'Canada': 'N/A', 'Europe': 'N/A', 'Australia/NZ': 'N/A', 'Other': 'N/A'}, '']
['', 'Brand new, never worn condition 10/10, Size L Oak Bogotá Bomber, This is the goat JE bomber, and the best Bogotá silhouette that John has designed, by far IMO., The colour and fit are both incredible, You won’t find another in this condition', 'Brown', 'New', 'Canada', {'United States': '$30', 'United Kingdom': 'N/A', 'Asia': 'N/A', 'C

KeyboardInterrupt: 